In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("category-RDD")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/25 08:47:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/25 08:47:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/25 08:47:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
raw_RDD = sc.parallelize(["id,item,cateogry,reviews",
    "0,짜장면,중식,125",
    "1,짬뽕,중식,235",
    "2,김밥,분식,32",
    "3,떡볶이,분식,534",
    "4,라멘,일식,223",
    "5,돈가스,일식,52",
    "6,우동,일식,12",
    "7,쌀국수,아시안,312",
    "8,햄버거,패스트푸드,12",
    "9,치킨,패스트푸드,23"])

In [6]:
header = raw_RDD.first()
filtered_lines = raw_RDD.filter(lambda row: row != header)
filtered_lines.take(2)

['0,짜장면,중식,125', '1,짬뽕,중식,235']

In [33]:
def parse(row):
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    return (category, reviews)

In [37]:
categoryReviews = filtered_lines.map(parse)
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [38]:
categoryReviewsCount = categoryReviews.mapValues(lambda x: (x, 1))
categoryReviewsCount.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [39]:
reduced = categoryReviewsCount.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [40]:
averages = reduced.mapValues(lambda x: x[0] / x[1])
averages.collect()